In [1]:
import sys
sys.path.append('C:\\Users\\dlfdus\\PycharmProjects\\mypacakge\\')
from corpus import DoublespaceLineCorpus
from corpus import DoublespaceLineCorpusWithList
from tqdm import tqdm_notebook
import requests
import pandas as pd
import re
import pickle
from collections import Counter

In [2]:
data = pd.read_csv('data/reviews/회사리뷰.csv')
positive = data["장점"]
negative = data["단점"]

In [3]:
data.head()

,1차산업군,2차산업군,회사명,날짜,직무,제목,장점,단점,원하는점
0,서비스업,호텔/여행/항공,(주)하나투어,2018/9/28,서비스/고객지원,"""배울점은 많고 업계1위라서 자부심도 높음 연봉복지모두 나쁘지 않음""",업계1위 어딜가든 하나투어다닌다고하면 인정함 배울것이 많음 작은여행사다니는 친구들에...,하나투어공채랑 그외 계열사에서 뽑은 인원들 차별이 존재함\r\r\n근데 근무는 같이...,보여지는것만에 치중하지말고 정말 직원들이 원하는걸 해줬으면 좋겠음\r\r\n시대에 ...
1,서비스업,호텔/여행/항공,(주)하나투어,2018/9/28,경영/기획/컨설팅,"""여행혜택,아직은 가족같은, 연차 자유롭게, 워라밸, 연봉은 타업종비해 낮으나 적응...","여행혜택, 가족같은, 연차 자유롭게 사용할수 있음. 가족들,아이들과의 삶의 균형 측...","타업종대비 급여는 낮음. 변화가 크지 않고, 항상 비슷한 패턴의 일의 반복임.\r\...","좋은 점은 살리고, 변화는 필요. 업종이 서비스업이라 타제조업대비 급여는 낮다는 것..."
2,서비스업,호텔/여행/항공,(주)하나투어,2018/9/26,영업/제휴,"""워라벨은 있는기업 젊은회사 연차 편하게씀""",연차는 편하게쓰는편 젊은회사라 커뮤니케이션은 잘되는편,직원들의 낮은수준 회사 규모에비해 직원들이 너무 공부안함,줄세우기말고 순환보직 필요 젊은회사인데 꼰대기질 있음
3,서비스업,호텔/여행/항공,(주)하나투어,2018/9/26,마케팅/시장조사,"""여행업에 대한 기본적인 이해가 가능하지만 발전할 수 없는 구조임을 느끼게해준 회사""","브랜드 인지도 1위, 저녁이 있는 삶, 여행갈 때 할인","직원들의 패배감(부산지사), 산업의 성장동력 약화, 워라벨을 즐길 수 없는 연봉",새로운 아이디어를 통해 기업이 발전할 수 있는 방향을 빠른 시일내에 마련해야한다
4,서비스업,호텔/여행/항공,(주)하나투어,2018/9/25,전문직,"""전문직을 너무 대우 안해줌\r\r\n1년 넘게 공부해서 자격증 따고 들어갔는데 너...",이름이 알려져 있음\r\r\n사람들이 어디다니냐 그러면 말했울때 모르지는 않음..ㅠ...,급여 너무 짜고 프로세스 전무에 자격증 있음에도 대우 못 받음...ㅠ,그냥 일한대로 주고 자격증 보고 뽑았으면 그에 맞는 대우를...


In [101]:
#데이터의 정규표현 제거

In [4]:
for idx, i in enumerate(tqdm_notebook(positive)):
    if '\r' in positive[idx]:positive[idx] = positive[idx].replace('\r','')
    if '\t' in positive[idx]:positive[idx] = positive[idx].replace('\t',' ')
    if '\n' in positive[idx]:positive[idx] = positive[idx].replace('\n',' ')

In [5]:
for idx, i in enumerate(tqdm_notebook(negative)):
    if '\r' in negative[idx]:negative[idx] = negative[idx].replace('\r','')
    if '\t' in negative[idx]:negative[idx] = negative[idx].replace('\t','')
    if '\n' in negative[idx]:negative[idx] = negative[idx].replace('\n',' ')

In [6]:
PN = list(positive) + list(negative)

In [7]:
# 알파벳, 특수문자 제거

In [25]:
normalize_ = False

In [9]:
if normalize_ == True:
    from soynlp.hangle import normalize
    normalized_data = [normalize(i, number = True, punctuation=False) for i in PN]
    with open('data/tmp/normalized_data.pkl', 'wb') as f:
        pickle.dump(normalized_data, f)
else:
    with open('data/tmp/normalized_data.pkl', 'rb') as f:
        normalized_data = pickle.load(f)

In [28]:
print(positive[5], '\n\n ↓ \n\n{}'.format(normalize(positive[5])))

연차사용은 비교적 자유로운편. 회사 복지-거점운영,시차근무,재택근무,직원용항공권제공, 각 상품 직원가이용 가능 . 팀바팀이지만 대체로 젊은 분위기. 정시퇴근함. 과장까지는 승진 어려움 없음. 

 ↓ 

연차사용은 비교적 자유로운편 회사 복지 거점운영 시차근무 재택근무 직원용항공권제공 각 상품 직원가이용 가능 팀바팀이지만 대체로 젊은 분위기 정시퇴근함 과장까지는 승진 어려움 없음


In [103]:
#CRF를 이용한 띄어쓰기 오류 교정

In [29]:
import pycrfsuite
from pycrfsuite_spacing import TemplateGenerator
from pycrfsuite_spacing import CharacterFeatureTransformer
from pycrfsuite_spacing import PyCRFSuiteSpacing

In [30]:
to_feature = CharacterFeatureTransformer(TemplateGenerator(begin=-2, 
                                                           end=2,
                                                           min_range_length=3,
                                                           max_range_length=3))

In [31]:
TrainCRF = False

In [32]:
if TrainCRF == True:
    correct = PyCRFSuiteSpacing(to_feature,
                                feature_minfreq=5, 
                                max_iterations=100,
                                l1_cost=0,
                                l2_cost=1.0
                               )
    correct.train(normalized_data, 'data/tmp/package_test.crfsuite')
else:
    model_path = 'data/tmp/package_test.crfsuite'
    correct = PyCRFSuiteSpacing(to_feature)
    correct.load_tagger(model_path)

In [36]:
sentence = normalize(positive[5])

In [34]:
correct('연차사용은 비교적 자유로운편 회사 복지 거점운영 시차근무 재택근무 직원용항공권제공 각 상품 직원가이용 가능 팀바팀이지만 대체로 젊은 분위기 정시퇴근함 과장까지는 승진 어려움 없음')

'연차 사용은 비교적 자유로운 편 회사 복지 거점 운영 시차 근무 재택근무 직원용 항공권 제공 각 상품 직원가 이용 가능 팀바팀이지만 대체로 젊은 분위기 정시퇴근함 과장까지는 승진 어려움 없음'

In [37]:
print(sentence, '\n\n ↓ \n\n{}'.format(correct(sentence)))

연차사용은 비교적 자유로운편 회사 복지 거점운영 시차근무 재택근무 직원용항공권제공 각 상품 직원가이용 가능 팀바팀이지만 대체로 젊은 분위기 정시퇴근함 과장까지는 승진 어려움 없음 

 ↓ 

연차 사용은 비교적 자유로운 편 회사 복지 거점 운영 시차 근무 재택근무 직원용 항공권 제공 각 상품 직원가 이용 가능 팀바팀이지만 대체로 젊은 분위기 정시퇴근함 과장까지는 승진 어려움 없음


In [39]:
spacing = False

In [40]:
if spacing == True:
    spaced_data = [correct(i) for i in normalized_data]
    with open('data/tmp/spaced_data.pkl', 'wb') as f:
        pickle.dump(spaced_data, f)
else:
    with open('data/tmp/spaced_data.pkl', 'rb') as f:
        spaced_data = pickle.load(f)

In [41]:
with open('data/tmp/spaced_data.txt', 'w', encoding='utf-8') as f:
    for i in spaced_data:
        f.write(i + '\n')

In [42]:
from soynlp.word import WordExtractor
from corpus import DoublespaceLineCorpus

corpus_fname = 'data/tmp/spaced_data.txt'
#corpus_fname = 'data/tmp/spaced_data.pkl'
corpus = DoublespaceLineCorpus(corpus_fname, iter_sent=False)
word_extractor = WordExtractor(left_max_length=10, right_max_length=6, min_count=5)
word_extractor.train(corpus)

spaced_corpus = list(corpus)

with open('data/tmp/spaced_corpus.pkl', 'wb') as f:
    pickle.dump(spaced_corpus, f)

training was done. used memory 0.536 Gb


In [43]:
with open('data/tmp/spaced_corpus.pkl', 'rb') as f:
    spaced_corpus = pickle.load(f)

In [44]:
for n_sent, sent in enumerate(corpus):
    if n_sent > 5: break
    print(n_sent, sent)

0 업계 1위 어딜가든 하나투어 다닌다고 하면 인정함 배울것이 많음 작은 여행사 다니는 친구들에 비해 연봉이든 복지든 훨씬 좋음

1 여행 혜택 가족같은 연차 자유롭게 사용할 수 있음 가족들 아이들과의 삶의 균형 측면에서 좋음

2 연차는 편하게 쓰는편 젊은 회사라 커뮤니케이션은 잘되는 편

3 브랜드 인지도 1위 저녁이 있는 삶 여행갈 때 할인

4 이름이 알려져 있음 사람들이 어디다 니냐 그러면 말했 울때 모르지는 않음 ㅠㅠㅠㅠ 근데 들어가서 의 업무는 매우 힘들면서도 보람을 찾기 너무 힘듬

5 연차 사용은 비교적 자유로운 편 회사 복지 거점 운영 시차 근무 재택근무 직원용 항공권 제공 각 상품 직원가 이용 가능 팀바팀이지만 대체로 젊은 분위기 정시퇴근함 과장까지는 승진 어려움 없음



In [45]:
#단어 점수 추출 LR 방식. 일반적으로 명사는 왼쪽, 조사는 명사의 오른쪽에 해당

In [46]:
scores = word_extractor.word_scores()

all cohesion probabilities was computed. # words = 77358
all branching entropies was computed # words = 95252
all accessor variety was computed # words = 95252


In [56]:
noun_extraction = False

In [57]:
from soynlp.noun import LRNounExtractor

if noun_extraction == True:
    noun_extractor = LRNounExtractor()
    #명사를 학습
    nouns = noun_extractor.train_extract(corpus)
    with open('data/tmp/nouns.pkl', 'wb') as f:
        pickle.dump(nouns, f)
else:
    with open('data/tmp/nouns.pkl', 'rb') as f:
        nouns = pickle.load(f)

In [58]:
nouns['복지']

NounScore(frequency=18878, score=0.7179762166701659, known_r_ratio=0.7652634961439588)

In [59]:
for word in ['연봉', '대기업','기업', '복지','높은','낮은','스트레스','높음','세다','워라밸','높은','보수적인', '중소기']:
    print('%s is noun? %r\n' % (word, word in nouns))

연봉 is noun? True

대기업 is noun? True

기업 is noun? True

복지 is noun? True

높은 is noun? False

낮은 is noun? False

스트레스 is noun? True

높음 is noun? True

세다 is noun? False

워라밸 is noun? True

높은 is noun? False

보수적인 is noun? False

중소기 is noun? False



In [60]:
#cohesion score 를 이용한 토크나이징

In [61]:
from soynlp.tokenizer import LTokenizer
calculate_cohesion_scores = False

In [62]:
if calculate_cohesion_scores == True:
    cohesion_scores = {word:score.cohesion_forward for word, score in scores.items()}
    ltokenizer = LTokenizer(scores=cohesion_scores)
    
    with open('data/tmp/jobplanet_cohesionscore.pkl', 'wb') as f:
        pickle.dump(cohesion_scores, f)
else:
    with open('data/tmp/jobplanet_cohesionscore.pkl', 'rb') as f:
        cohesion_scores = pickle.load(f)
    ltokenizer = LTokenizer(scores=cohesion_scores)

In [63]:
ltokenizer.tokenize('높은 업무강도 가족적인 분위기라고는 하지만 실제로는 군대문화')

['높은', '업무', '강도', '가족적인', '분위기', '라고는', '하지만', '실제로', '는', '군대', '문화']

In [64]:
sentence = correct(sentence)

In [65]:
print(sentence, '\n\n ↓ \n\n{}'.format(ltokenizer.tokenize(sentence)))

연차 사용은 비교적 자유로운 편 회사 복지 거점 운영 시차 근무 재택근무 직원용 항공권 제공 각 상품 직원가 이용 가능 팀바팀이지만 대체로 젊은 분위기 정시퇴근함 과장까지는 승진 어려움 없음 

 ↓ 

['연차', '사용', '은', '비교적', '자유로운', '편', '회사', '복지', '거점', '운영', '시차', '근무', '재택근무', '직원', '용', '항공', '권', '제공', '각', '상품', '직원', '가', '이용', '가능', '팀바팀이지만', '대체로', '젊은', '분위기', '정시퇴근', '함', '과장까지는', '승진', '어려움', '없음']


In [66]:
word_counter = Counter(word for text in spaced_data for word in text.split())

In [67]:
sorted(word_counter.items(), key=lambda x:x[1], reverse=True)[:10]

[('수', 31747),
 ('있음', 26623),
 ('업무', 14391),
 ('회사', 13707),
 ('없음', 13655),
 ('많이', 11319),
 ('있는', 11132),
 ('많음', 10406),
 ('있다', 10099),
 ('잘', 9545)]

In [68]:
n_words_before_pruning = len(word_counter)

min_count = 10
word_dictionary = {word:freq for word,freq in word_counter.items() if freq >= min_count and len(word) > 1}
n_words_after_pruning  = len(word_dictionary)

print('%d --> %d' % (n_words_before_pruning, n_words_after_pruning))

183527 --> 22485


In [69]:
save_word_dictionary = False

In [70]:
if save_word_dictionary == True:
    with open('data/tmp/word_dictionary.pkl', 'wb') as f:
        pickle.dump(word_dictionary, f)
else:
    with open('data/tmp/word_dictionary.pkl', 'rb') as f:
        word_dictionary = pickle.load(f)

In [71]:
tokenize = False

In [72]:
if tokenize == True:
    tokenized_reviews = []
    for review in tqdm_notebook(spaced_data):
        tokenized = ''
        for word in ltokenizer.tokenize(review):
            if word in word_dictionary and len(word)>1:
                #단어사전에 있는 단어들과 2음절 이상의 단어들만 토크나이징
                tokenized += word
                tokenized += ' '
        tokenized = tokenized.strip()
        tokenized_reviews.append(tokenized)
    with open('data/tmp/tokenized_reviews.pkl', 'wb') as f:
        pickle.dump(tokenized_reviews, f)
else:
    with open('data/tmp/tokenized_reviews.pkl', 'rb') as f:
        tokenized_reviews = pickle.load(f)

In [73]:
tokenized_reviews[:5]

['업계 1위 다닌다고 하면 인정 배울 것이 많음 작은 여행사 다니는 친구들 비해 연봉 이든 복지 훨씬 좋음',
 '여행 혜택 가족같은 연차 자유롭게 사용 있음 가족 아이들과 삶의 균형 측면 에서 좋음',
 '연차 편하게 쓰는 젊은 회사 커뮤니케이션',
 '브랜드 인지도 1위 저녁 있는 여행 할인',
 '이름이 알려져 있음 사람들이 어디 그러면 지는 않음 ㅠㅠ ㅠㅠ 근데 들어 가서 업무 매우 힘들 보람을 찾기 너무 힘듬']

In [74]:
# 토크나이징하여 분리해낸 단어들 중에서 명사만을 추리는 함수
def get_noun(doc, nouns=nouns):
    a = ltokenizer.tokenize(doc)
    noun_list = []
    for indx, i in enumerate(a):
        if i in nouns:
            noun_list.append(i)
    return noun_list

In [78]:
get_noun('높은 업무강도 가족적인 분위기라고는 하지만 실제로는 군대문화')

['업무', '강도', '분위기', '군대', '문화']

In [80]:
sentence = '연차 사용은 비교적 자유로운 편 회사 복지 거점 운영 시차 근무 재택근무 직원용 항공권 제공 각 상품 직원가 이용 가능 팀바팀이지만 대체로 젊은 분위기 정시퇴근함 과장까지는 승진 어려움 없음 '

In [81]:
print(sentence, '\n\n ↓ \n\n{}'.format(get_noun(sentence)))

연차 사용은 비교적 자유로운 편 회사 복지 거점 운영 시차 근무 재택근무 직원용 항공권 제공 각 상품 직원가 이용 가능 팀바팀이지만 대체로 젊은 분위기 정시퇴근함 과장까지는 승진 어려움 없음  

 ↓ 

['연차', '사용', '비교적', '회사', '복지', '거점', '운영', '시차', '근무', '재택근무', '직원', '제공', '상품', '직원', '이용', '가능', '분위기', '정시퇴근', '승진', '어려움', '없음']


In [79]:
noun_data = [get_noun(i) for i in tokenized_reviews]

In [100]:
with open('data/tmp/tokenized_noun_reviews.pkl', 'wb') as f:
    pickle.dump(noun_data, f)

In [80]:
noun_data[:5]

[['업계', '1위', '하나투어', '인정', '많음', '여행사', '친구들', '연봉', '복지', '좋음'],
 ['여행', '혜택', '연차', '사용', '있음', '가족', '균형', '측면', '좋음'],
 ['연차', '회사', '커뮤니케이션'],
 ['브랜드', '1위', '저녁', '여행', '할인'],
 ['있음', '업무', '찾기', '힘듬']]

In [81]:
with open('data/tmp/doc2vec_word_data.txt', 'w') as f:
    for idx, i in enumerate(noun_data):
        #장점
        if idx < data.shape[0]:
            f.write(data.loc[idx, '회사명'] + '\t')
            for j in i:
                f.write(j + ' ')
            f.write('\n')
        #단점
        else:
            f.write(data.loc[idx-data.shape[0], '회사명'] + '\t')
            for j in i:
                f.write(j + ' ')
            f.write('\n')

In [83]:
corpus_ = open('data/tmp/doc2vec_word_data.txt', 'r')
line = corpus_.readlines()

In [87]:
line

['(주)하나투어\t업계 1위 하나투어 인정 많음 여행사 친구들 연봉 복지 좋음 \n',
 '(주)하나투어\t여행 혜택 연차 사용 있음 가족 균형 측면 좋음 \n',
 '(주)하나투어\t연차 회사 커뮤니케이션 \n',
 '(주)하나투어\t브랜드 1위 저녁 여행 할인 \n',
 '(주)하나투어\t있음 업무 찾기 힘듬 \n',
 '(주)하나투어\t연차 사용 비교적 회사 복지 거점 운영 시차 근무 재택근무 직원 제공 상품 직원 이용 가능 분위기 정시퇴근 승진 어려움 없음 \n',
 '(주)하나투어\t국내 여행사 확실 업무 장점 메이저 회사 네임 회사 이직 \n',
 '(주)하나투어\t업무 균형 확실 여행 관련 복지 사람들 훌륭 직장 \n',
 '(주)하나투어\t중견기업 기업 연차 있음 야근 필요 경우 성과 보상 출퇴근 유연근무제 \n',
 '(주)하나투어\t연차 육아휴직 좋음 재택근무 있음 \n',
 '(주)하나투어\t육아휴직 제도 여성 복지 있음 해외출장 많기 추천 \n',
 '(주)하나투어\t1위 기업 자부심 뿌듯 \n',
 '(주)하나투어\t연차 복지 눈치 없지 있음 \n',
 '(주)하나투어\t업계 1위 기업 만족 여행업계 다운 연차 사용 직원 휴가 여행 지원 많음 부서 부서 존재 \n',
 '(주)하나투어\t연차 육아휴직 있음 거점 근무 재택근무 신입 거점 근무 경우 가능 재택근무 거의 과장 이상 가능 교육 괜춘 휴가 괜춘 부서 칼퇴 했음 \n',
 '(주)하나투어\t눈치 연차 대리급 이상 \n',
 '(주)하나투어\t여행사 캐주얼 분위기 자유 복장 남여 비율 균등 성향 직장 일것 \n',
 '(주)하나투어\t연차 사용가능 육아 휴직 사용 거점 근무 높음 야근 홍보 영업 상품 거의 없음 회식 업무 생각 경향 때문 대리 대부분 캐쥬얼 데이 복장 생각 \n',
 '(주)하나투어\t연차 소진 99 99 99 기회 많음 보장 기업문화 \n',
 '(주)하나투어\t연차 그외 조직문화 복장 문제 되지 \n',
 '(주)하나투어\t대기업 문화 재택근무 근무 가능 복지 

reviews = []
for idx, i in enumerate(line):
    temp = i.split('\t')[1]
    temp = re.sub('\n', '', temp)
    reviews.append(temp)

In [88]:
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

def TaggedReview(reviews):
    for doc in reviews:
        company_tag, review = doc.split('\t')
        text = re.sub('\n', '', review)
        yield TaggedDocument(words=text.split(), tags=['Company_%s' % company_tag])

doc2vec_corpus = list(TaggedReview(line))

C:\Users\dlfdus\Anaconda3\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [89]:
for num_doc, doc in enumerate(doc2vec_corpus):
    if num_doc > 5: break
    print(doc)

TaggedDocument(['업계', '1위', '하나투어', '인정', '많음', '여행사', '친구들', '연봉', '복지', '좋음'], ['Company_(주)하나투어'])
TaggedDocument(['여행', '혜택', '연차', '사용', '있음', '가족', '균형', '측면', '좋음'], ['Company_(주)하나투어'])
TaggedDocument(['연차', '회사', '커뮤니케이션'], ['Company_(주)하나투어'])
TaggedDocument(['브랜드', '1위', '저녁', '여행', '할인'], ['Company_(주)하나투어'])
TaggedDocument(['있음', '업무', '찾기', '힘듬'], ['Company_(주)하나투어'])
TaggedDocument(['연차', '사용', '비교적', '회사', '복지', '거점', '운영', '시차', '근무', '재택근무', '직원', '제공', '상품', '직원', '이용', '가능', '분위기', '정시퇴근', '승진', '어려움', '없음'], ['Company_(주)하나투어'])


In [90]:
train_Doc2Vec = False

In [91]:
%%time
doc2vec_fname = 'data/tmp/doc2vec_model.pkl' 
if train_Doc2Vec:
    doc2vec_model = Doc2Vec(doc2vec_corpus, size=200, alpha=0.05, window=5, min_count=5, negative=5) 
    with open(doc2vec_fname, 'wb') as f:
        pickle.dump(doc2vec_model, f)

else:
    with open(doc2vec_fname, 'rb') as f:
        doc2vec_model = pickle.load(f)

Wall time: 247 ms


In [95]:
P_doc2vec_corpus = doc2vec_corpus[:int(len(doc2vec_corpus)/2)]
N_doc2vec_corpus = doc2vec_corpus[int(len(doc2vec_corpus)/2):]

In [98]:
train_Doc2Vec_PN = False

In [99]:
%%time
P_doc2vec_fname = 'data/tmp/P_doc2vec_model.pkl' 
N_doc2vec_fname = 'data/tmp/N_doc2vec_model.pkl' 

if train_Doc2Vec_PN:
    P_doc2vec_model = Doc2Vec(P_doc2vec_corpus, size=200, alpha=0.05, window=5, min_count=5, negative=5) 
    with open(P_doc2vec_fname, 'wb') as f:
        pickle.dump(P_doc2vec_model, f)
    N_doc2vec_model = Doc2Vec(N_doc2vec_corpus, size=200, alpha=0.05, window=5, min_count=5, negative=5) 
    with open(N_doc2vec_fname, 'wb') as f:
        pickle.dump(N_doc2vec_model, f)

else:
    with open(P_doc2vec_fname, 'rb') as f:
        P_doc2vec_model = pickle.load(f)
    with open(N_doc2vec_fname, 'rb') as f:
        N_doc2vec_model = pickle.load(f)

Wall time: 2min
